In [ ]:
from abc import ABC, abstractclassmethod, abstractproperty
from datetime import datetime

class ContaIterador:
  def __init__(self,conta):
    self.__conta = conta
    self.index = 0

  def __iter__(self):
    return self

  def __next__(self):
    try:
      conta = self.contas[self.index]
      return f"""\
      Agênncia:\t{conta.agencia}
      Número:\t\t{conta.numero}
      Títular:\t{conta.cliente.nome}
      Saldo:\t\t{conta.saldo}
      """
    except IndexError:
      raise StopIteration
    self.index += 1

class Cliente:
  def __init__(self, endereco):
    self.endereco = endereco
    self.contas = []

  def realizar_transacao(self, conta, transacao):
    if len(conta.historico.transacoes) >= 10:
      print("Limite de transações atingido")
      return
    transacao.registrar(conta)

  def adicionar_conta(self, conta):
    self.contas.append(conta)

class PessoaFisica(Cliente):
  def __init__(self, nome, data_nascimento, cpf, endereco):
    super().__init__(endereco)
    self.nome = nome
    self.data_nascimento = data_nascimento
    self.cpf = cpf

class PessoaJuridica(Cliente):
  def __init__(self, nome, cnpj, endereco):
    super().__init__(endereco)
    self.nome = nome
    self.cnpj = cnpj

class Conta:
  def __init__(self, numero, cliente):
    self._saldo = 0
    self._numero = numero
    self._agencia = "0001"
    self._cliente = cliente
    self._historico = Historico()
  @classmethod
  def nova_conta(cls, cliente, numero):
    return cls(numero, cliente)

  @property
  def saldo(self):
    return self._saldo

  @property
  def numero(self):
    return self._numero

  @property
  def agencia(self):
    return self._agencia

  @property
  def cliente(self):
    return self._cliente

  @property
  def historico(self):
    return self._historico

  def sacar(self, valor):
    saldo = self.saldo
    excedeu_saldo = valor > saldo
    if excedeu_saldo:
      print("Saldo insuficiente")
    elif saldo > 0:
      saldo -= valor
      print(f"Foram sacadas R${valor:.2f} e o saldo é de R${valor:.2f}")
      return True
    else:
      print("Valor inválido")
    return False

  def depositar(self, valor):
    saldo = self.saldo
    if valor > 0:
      saldo += valor
      print(f" Foram depoisitado R${valor:.2f} e o eu saldo é de R${saldo:.2f}")
    else:
      print("Valor inválido")
      return False
    return True

class ContaCorrente(Conta):
  def __init__(self, numero, cliente, limite=500, LIMITES_SAQUES=3):
    super().__init__(numero, cliente)
    self.limite = limite
    self.LIMITES_SAQUES = LIMITES_SAQUES

  def sacar (self, valor):
    numero_saques = len([transacao for transacao in self.historico.transacoes if transacao["tipo"] == Saque.__name__])
    excedeu_limite = valor > self.limite
    excedeu_saques = numero_saques >= self.LIMITES_SAQUES
    if excedeu_limite:
      print("Limite de saque excedido")
    elif excedeu_saques:
      print("Limite de saques atingido")
    else:
      return super().sacar(valor)
    return False

  def __str__(self):
    return f"""\
      Agência:\t{self.agencia}
      C/C:\t{self.numero}
      Titular:\t{self.cliente.nome}
    """

class Historico:
  def __init__(self):
    self._transacoes = []

  @property
  def transacoes(self):
    return self._transacoes

  def adicionar_trasacao(self, transacao):
    self._transacoes.append(
        {
          "tipo": transacao.__class__.__name__,
          "valor": valor,
          "data": datetime.now().strftime("%d-%m-%Y %H:%M:%S")
        }
    )
  def gerar_relatorio(self, tipo_transacao):
    for transacao in self._transacoes:
      if tipo_transacao is None or transacao["tipo"].lower() == tipo_transacao.lower():
        yield transacao

class Transacao(ABC):
  @property
  @abstractproperty
  def valor(self):
    pass
  @abstractclassmethod
  def registrar(self, conta):
    pass

class Deposito(Transacao):
  def __init__(self, valor):
    self._valor = valor
  @property
  def valor(self):
    return self._valor
  def registrar(self, conta):
    conta.historico.adicionar_transacao(self)

class Saque(Transacao):
  def __init__(self, valor):
    self._valor = valor
  @property
  def valor(self):
    return self._valor
  def registrar(self, conta):
    conta.historico.adicionar_transacao(self)

def filtrar_usuario(cpf, usuarios):
  usuarios_filtrados = [usuario for usuario in usuarios if usuario.cpf == cpf]
  return usuarios_filtrados[0] if usuarios_filtrados else None

def validar_cpf(cpf):
    if not cpf.isdigit() or len(cpf) != 11:
        return False
    if cpf == cpf[0] * 11:
        return False
    soma1 = sum(int(cpf[i]) * (10 - i) for i in range(9))
    digito1 = (soma1 * 10 % 11) % 10
    soma2 = sum(int(cpf[i]) * (11 - i) for i in range(10))
    digito2 = (soma2 * 10 % 11) % 10
    return cpf[-2:] == f"{digito1}{digito2}"

def log_transacao(func):
  def envelope(*args, **kwargs):
    resultado = func(*args, **kwargs)
    print(f"{datetime.now()}: {func.__name__.upper()}")
    return resultado
  return envelope

@log_transacao
def depositar(usuarios):
  cpf = input("Digite seu CPF (apenas números): ")
  usuario = filtrar_usuario(cpf, usuarios)
  if not usuario:
    print("Usuário não encontrado. Cadastre-se primeiro")
    return
  valor = float(input("Digite o valor do depósito: "))
  transacao = Deposito(valor)
  conta = recuperar_conta_usuario(usuario)
  if not conta:
    return
  usuario.realizar_transacao(conta, transacao)
  print("Depósito realizado com sucesso")

@log_transacao
def sacar(usuarios):
  cpf = input("Digite seu CPF (apenas números): ")
  usuario = filtrar_usuario(cpf, usuarios)
  if not usuario:
    print("Usuário não encontrado. Cadastre-se primeiro")
    return
  valor = float(input("Digite o valor do saque: "))
  transacao = Saque(valor)
  conta = recuperar_conta_usuario(usuario)
  if not conta:
    return
  usuario.realizar_transacao(conta, transacao)
  print("Saque realizado com sucesso")

def recuperar_conta_usuario(usuario):
  if not usuario.contas:
    print("Usuário não possui conta")
    return
  return usuario.contas[0]

@log_transacao
def exibir_extrato(clientes):
  cpf = input("Digite seu CPF (apenas números): ")
  usuario = filtrar_usuario(cpf, clientes)
  if not usuario:
    print("Usuário não encontrado. Cadastre-se primeiro")
    return
  conta = recuperar_conta_usuario(usuario)
  if not conta:
    return
  print("\n================ EXTRATO ================")
  extrato = ""
  tem_transacao = False
  if not tem_transacao:
    extrato = "Não foram realizadas movimentações."
  else:
    for transacao in conta.hstorico.gerar_relatorio():
      extrato += f"\n {transacao['data']}\n{transacao['tipo']}:\n\tR${transacao['valor']:.2f}"
  print(extrato)
  print(f"\nSaldo:\n\tR${conta.saldo:.2f}")
  print("==========================================")

@log_transacao
def criar_conta(numero_conta, clientes, contas):
  cpf = input("Digite seu CPF (apenas números): ")
  cliente = filtrar_usuario(cpf, clientes)
  if not cliente:
    print("Usuário não encontrado. Cadastre-se primeiro")
    return
  conta = ContaCorrente.nova_conta(cliente=cliente, numero=numero_conta)
  cliente.contas.append(conta)
  contas.append(conta)
  print("Conta criada com sucesso")

def listar_contas(contas):

  for conta in ContaIterador(contas):
    print("=" * 100)
    print(textwrap.dedent(str(conta)))

@log_transacao
def criar_usuario(usuarios):
  cpf = input("Digite seu CPF (apenas números): ")
  usuario = filtrar_usuario(cpf, usuarios)
  if usuario:
    print("Usuário já cadastrado")
    return
  elif not validar_cpf(cpf):
    print("CPF inválido")
    return
  nome = input("Digite seu nome: ")
  data_nascimento = input("Digite sua data de nascimento (no formato xx/xx/xxxx): ")
  endereco = input("Digite seu endereço (formato: logradouro, número, bairro, cidade/UF ): ")
  partes = [parte.strip() for parte in endereco.split(',')]
  if len(partes) != 4:
    print("Endereço inválido! Use o formato: logradouro, número, bairro, cidade/UF")
    return
  usuario = PessoaFisica(nome=nome, data_nascimento=data_nascimento, cpf=cpf, endereco=endereco)
  usuarios.append(usuario)
  print("Cliente criado com sucesso")

def main():
  usuarios =[]
  contas = []

  menu = """

  [d] Depositar
  [s] Sacar
  [e] Extrato
  [u] Criar Usuário
  [c] Criar Conta
  [l] Listar Contas
  [q] Sair

  => """

  while True:
    opcao = input(menu)
    if opcao == "d":
      depositar(usuarios)

    elif opcao == "s":
      sacar(usuarios)

    elif opcao == "e":
      exibir_extrato(usuarios)

    elif opcao == "u":
      criar_usuario(usuarios)
    elif opcao == "c":
      criar_conta(len(contas) + 1, usuarios, contas)
    elif opcao == "l":
      print("Listar Contas")
      listar_contas(contas)

    elif opcao == "q":
        print("Saindo...")
        break
    else:
        print("Operação inválida, por favor selecione novamente a operação desejada.")

main()



  [d] Depositar
  [s] Sacar
  [e] Extrato
  [u] Criar Usuário
  [c] Criar Conta
  [l] Listar Contas
  [q] Sair

  => u


KeyboardInterrupt: Interrupted by user